Adapted from [Willie Wheeler's single-queue network simulation](https://medium.com/wwblog/simulating-an-m-m-1-queue-in-python-f894f5a68db2).

In [ ]:
# Imports
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from IPython.display import Markdown

In [ ]:
# Constants, etc.
rng = np.random.default_rng(34)

TITLE_SIZE = 18
HIST_BINS = 51

## Queue structure

The system is an open queue system.

Arrival -> =====](Q1) -> =====](Q2) -> Departure

Little's Law:
$$ {\mathbb E}[N] = \lambda {\mathbb E}[T] $$

In [ ]:
# Arrival
# Arrival time is uniform, interarrival time has boundaries of [1, 5] (mean time of 3s)

# Queue Q1
# Service time is exponential, service rate m1 = 0.5/s (mean time of 2s)

# Queue Q2
# Service time is uniform, with boundaries of [1, 3] (mean time of 2s)

In [ ]:
def create_params(num_customers: int, mean_interarrival_time: float, mean_interarrival_range: float, mean_service_time_1: float, mean_service_time_2: float, mean_service_time_2_range: float):
    return {
        "n": num_customers,
        "mean_interarrival_time": mean_interarrival_time,
        "mean_interarrival_low": mean_interarrival_time - mean_interarrival_range/2,
        "mean_interarrival_high": mean_interarrival_time + mean_interarrival_range/2,
        "mean_service_time_1": mean_service_time_1,
        "mean_service_time_2": mean_service_time_2,
        "mean_service_time_2_low": mean_service_time_2 - mean_service_time_2_range/2,
        "mean_service_time_2_high": mean_service_time_2 + mean_service_time_2_range/2,
        "mean_arrival_rate": 1.0 / mean_interarrival_time,
        "mean_service_rate_1": 1.0 / mean_service_time_1,
        "mean_service_rate_2": 1.0 / mean_service_time_2,
        "num_bins": int(num_customers * mean_interarrival_time)
    }

In [19]:
def build_customers_dataframe(params, interarrival_times, arrival_times, service_1_times, service_2_times):
    n = params["n"]

    customers_df = pd.DataFrame({
        "interarrival_time": interarrival_times,
        "arrival_time": arrival_times, # When the customer arrives in the system
        "service_time_1": service_1_times, # How long the customer is serviced in service 1
        "service_time_2": service_2_times, # How long the customer is serviced in service 2
        "start_time_1": np.zeros(n), # When the customer starts being serviced in service 1
        "depart_time_1": np.zeros(n), # When the customer is done being serviced by service 1
        "start_time_2": np.zeros(n), # When the customer starts being serviced in service 2
        "depart_time_2": np.zeros(n), # When the customer departs from the system (aka done being serviced by service 2)
        "response_time": np.zeros(n), # How long the customer spends within the system
        "wait_time_1": np.zeros(n), # How long the customer waits in queue 1
        "wait_time_2": np.zeros(n), # How long the customer waits in queue 2
    })

    # The first customer is the easiest to document
    customers_df.loc[0, "start_time_1"] = customers_df.loc[0, "arrival_time"]
    customers_df.loc[0, "depart_time_1"] = customers_df.loc[0, "start_time_1"] + customers_df.loc[0, "service_time_1"]
    customers_df.loc[0, "start_time_2"] = customers_df.loc[0, "depart_time_1"]
    customers_df.loc[0, "depart_time_2"] = customers_df.loc[0, "start_time_2"] + customers_df.loc[0, "service_time_2"]

    # Each customer has to wait for the previous customer(s) to finish their stuff first
    for i in range(1, n):
        # Customer i can only start when customer i-1 is done with the service at hand
        customers_df.loc[i, "start_time_1"] = max(customers_df.loc[i, "arrival_time"], customers_df.loc[i-1, "depart_time_1"])
        customers_df.loc[i, "depart_time_2"] = customers_df.loc[i, "start_time_1"] + customers_df.loc[i-1, "service_time_1"]
        customers_df.loc[i, "start_time_2"] = max(customers_df.loc[i, "depart_time_1"], customers_df.loc[i-1, "depart_time_2"])
        customers_df.loc[i, "depart_time_2"] = customers_df.loc[i, "start_time_2"] + customers_df.loc[i-1, "service_time_2"]

    customers_df["response_time"] = customers_df["depart_time_2"] - customers_df["arrival_time"]
    customers_df["wait_time_1"] = customers_df["start_time_1"] - customers_df["arrival_time"]
    customers_df["wait_time_2"] = customers_df["start_time_2"] - customers_df["depart_time_1"]

    return customers_df    

In [ ]:
def run_simulation(params):
    # Extracting parameters
    n = params["n"]
    iat = params["mean_interarrival_time"]
    ial = params["mean_interrarival_low"]
    iah = params["mean_interarrival_high"]
    st1 = params["mean_service_time_1"]
    st2 = params["mean_service_time_2"]
    st2l = params["mean_service_time_2_low"]
    st2h = params["mean_service_time_2_high"]

    # Sim data
    interarrival_times = rng.uniform(low=ial, high=iah, size=n)
    arrival_times = np.cumsum(interarrival_times)
    service_1_times = rng.exponential(scale=st1, size=n)
    service_2_times = rng.uniform(low=st2l, high= st2h, size=n)

    customers_df = build_customers_dataframe(params, interarrival_times, arrival_times, service_1_times, service_2_times)

    return customers_df